# SAPT Methods Testing

This notebook is for testing the workflow of the TrajectorySAPT object in MDSAPT. TrajectorySAPT is a subclass of AnalysisBase in MDAnalysis and iterates over an MD trajectory calculating interaction energies using SAPT for the specified pairs. This notebook just statically looks at one interaction pair in one frame.

In [21]:
import MDAnalysis as mda

from rdkit import Chem

from MDSAPT.mdsapt.reader import InputReader
from MDSAPT.mdsapt.optimizer import Optimizer, get_spin_multiplicity
from MDSAPT.mdsapt.viewer import Viewer

In [22]:
InSettings = InputReader('test_input.yaml')

Opt = Optimizer(InSettings)

View = Viewer(InSettings)

/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8

### 1. Setup

SAPT calculates the interaction energy between a dimer of molecules. When selected from a polypeptide backbone residues are not ready for SAPT as they are missing protons and have an unbalanced spin state. Optimizer solves that issue by capping the C terminus with a proton and replacing the missing protons on the N terminus.

The optimized pair is shown below (must run notebook for widget to work).

In [23]:
View.view_interaction_pair(11, 199)

NGLWidget(max_frame=97)

Once optimizer fixes the aforementioned issues this is residues before starting the SAPT run.

In [24]:
system = mda.Universe(InSettings.top_path, InSettings.trj_path)

resid1 = system.select_atoms('resid 11')
resid2 = system.select_atoms('resid 199')

View.view_optimized_interaction_pair(11, 199)

/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8

NGLWidget()

### 2. Generate Psi4 Inputs for each residue

with both residues selected the coordinates must be set up for Psi4. This means getting their charge and spin state as well as coordinates and atom names formatted into a string. 

In [25]:
from MDAnalysis.converters.RDKit import atomgroup_to_mol

def get_psi_mol(key: int, optimizer: Optimizer, residue: mda.AtomGroup) -> str:
        resid: mda.AtomGroup = optimizer.rebuild_resid(key, residue)
        rd_mol = atomgroup_to_mol(resid)

        coords: str = f'{Chem.GetFormalCharge(rd_mol)} {get_spin_multiplicity(rd_mol)}'
        for atom in resid.atoms:
            coords += f'\n{atom.name[0]} {atom.position[0]} {atom.position[1]} {atom.position[2]}'
        return coords

In [26]:
res1_in = get_psi_mol(11, Opt, resid1)
res2_in = get_psi_mol(199, Opt, resid2)

sapt_in = res1_in + '\n--\n' + res2_in + '\nunits angstrom'

print(sapt_in)

/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/alia/opt/anaconda3/envs/mdsapt2/lib/python3.8

1 1
N -4.013999938964844 3.3259999752044678 6.854000091552734
H -5.110000133514404 3.0179998874664307 7.247000217437744
H -4.09499979019165 3.000999927520752 5.697000026702881
H -3.364000082015991 2.805999994277954 7.296000003814697
C -3.7980000972747803 4.765999794006348 6.86299991607666
H -4.418000221252441 5.127999782562256 7.614999771118164
C -2.2730000019073486 4.854000091552734 7.360000133514404
H -1.5490000247955322 4.394999980926514 6.642000198364258
H -2.180000066757202 4.193999767303467 8.267000198364258
H -1.9869999885559082 5.919000148773193 7.538000106811523
C -4.107999801635742 5.644999980926514 5.64900016784668
O -5.047999858856201 6.392000198364258 5.74399995803833
H -3.485483169555664 5.59555196762085 4.709632396697998
--
1 1
N -4.447000026702881 11.470000267028809 8.847999572753906
H -4.551000118255615 11.736000061035156 7.677000045776367
H -4.063000202178955 10.336000442504883 8.718000411987305
H -3.7200000286102295 12.062999725341797 9.204000473022461
C -5.688000202

### 3. Run SAPT calculation

With the corrections finished the input string can be converted into a psi4 geometry and used to get the interaction energy. That calculations setting are specified in `test_input.yaml` and applied here. The output is given in `sapt_11-119_1.out`.

In [28]:
import psi4

dimer = psi4.geometry(sapt_in)
psi4.set_options(InSettings.sapt_settings['settings'])
psi4.set_memory(InSettings.memory)
psi4.set_num_threads(InSettings.ncpus)
psi4.set_output_file('sapt_11-199_1.out')


  Memory set to  11.176 GiB by Python driver.
  Threads set to 4 by Python driver.


In [30]:
sapt = psi4.energy(InSettings.sapt_settings['basis'], molecule=dimer)
print(sapt)

-608.1320412353217
